**Complex words = Words having more than 6 characters**

In [1]:
from google.colab import drive

drive.mount("/content/gdrive") 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import warnings 
warnings.filterwarnings("ignore") 

In [3]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
import numpy as np
import gensim
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re 
from bs4 import BeautifulSoup
import csv
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
df = pd.read_csv("/content/gdrive/MyDrive/newpro/MWP_indices_lastexperiment/data.csv")

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
df = df.iloc[:,0:2]
df

,Problem,Level
0,There are constants alpha and beta suc...,Level 5
1,The length of the segment between the points ...,Level 5
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5
3,"For some constants a and b, let [f(x) = ...",Level 5
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2
...,...,...
1739,What is the slope of the line passing through ...,Level 2
1740,Evaluate i^{11} + i^{16} + i^{21} + i^{26} + ...,Level 4
1741,If a is a constant such that 4x^2 - 12x + a...,Level 3
1742,"Given a star b = a^2 + 2ab + b^2 , what is ...",Level 3


In [8]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [9]:
df.iloc[:,1].replace([1, 2, 3, 4, 5],
                        ["Verylow", "Low", "Moderate", "High", "Veryhigh"], inplace=True)

In [10]:
df.head()

,Problem,Level
0,There are constants alpha and beta suc...,Veryhigh
1,The length of the segment between the points ...,Veryhigh
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Veryhigh
3,"For some constants a and b, let [f(x) = ...",Veryhigh
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low


In [11]:
df.nunique()

Problem     1744
Level          5
dtype: int64

In [12]:
df = df.iloc[:,0:2]
df

,Problem,Level
0,There are constants alpha and beta suc...,Veryhigh
1,The length of the segment between the points ...,Veryhigh
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Veryhigh
3,"For some constants a and b, let [f(x) = ...",Veryhigh
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low
...,...,...
1739,What is the slope of the line passing through ...,Low
1740,Evaluate i^{11} + i^{16} + i^{21} + i^{26} + ...,High
1741,If a is a constant such that 4x^2 - 12x + a...,Moderate
1742,"Given a star b = a^2 + 2ab + b^2 , what is ...",Moderate


Function to count number of sentences in the problem.

In [13]:
def count_sentences(df_col):
  number_of_sentences = sent_tokenize(df_col)
  return len(number_of_sentences)

In [14]:
df['sent_sum'] = df['Problem '].apply(count_sentences)

In [15]:
df.head(6)

,Problem,Level,sent_sum
0,There are constants alpha and beta suc...,Veryhigh,2
1,The length of the segment between the points ...,Veryhigh,2
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Veryhigh,5
3,"For some constants a and b, let [f(x) = ...",Veryhigh,3
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,1
5,What is the number of units in the area of the...,High,3


**Reading the math vocab data**

In [16]:
df_mwl = pd.read_csv("/content/gdrive/MyDrive/newpro/MWP_indices_lastexperiment/mathwords.csv")

In [17]:
df.shape

(1744, 3)

In [18]:
df_mwl.shape

(2258, 3)

In [19]:
df_mwl.head()

,Unnamed: 0,Word,Value
0,0,x,2675
1,1,What,597
2,2,frac,573
3,3,f,544
4,4,b,518


In [20]:
mwl = df_mwl.iloc[:,1:2]
mwl.head()

,Word
0,x
1,What
2,frac
3,f
4,b


Created list of all math vocabulary.

In [21]:
list = mwl['Word'].tolist()

print("the list of a single column from the dataframe\n",
        list,
        "\n",
        type(list))

the list of a single column from the dataframe
 ['x', 'What', 'frac', 'f', 'b', 'value', 'real', 'If', 'The', 'sqrt', 'A', 'c', 'sum', 'Find', 'n', 'answer', 'many', 'p', 'two', 'draw', 'Express', 'integers', 'equation', 'B', 'label', 'positive', 'graph', 'g', 'number', 'TRUE', 'numbers', 'How', 'Arrows', 'asy', 'integer', 'point', 'Let', 'r', 'form', 'C', 'text', 'cdot', 'xmax', 'right', 'left', 'values', 'xmin', 'points', 'k', 'square', 'sequence', 'size', 'fraction', 'line', 'ymin', 'ymax', 'one', 'xright', 'ytop', 'three', 'common', 'z', 'first', 'term', 'find', 'end', 'possible', 'dot', 'Ticks', 'product', 'Evaluate', 'linewidth', 'function', 'terms', 'expression', 'P', 'begin', 'solutions', 'For', 'axisarrowsize', 'axispen', 'xleft', 'ybottom', 'pen', 'cm', 'feet', 'gx', 'gy', 'area', 'ax', 'times', 'E', 'black', 'per', 'q', 'h', 'coordinates', 'Solve', 'total', 'FALSE', 'TicksArrx', 'TicksArry', 'extend', 'pTick', 'lfloor', 'rfloor', 'following', 'units', 'hours', 'Size', 'D', '

Count the words which are having more than 6 alphabets.

In [22]:
def countWords(mwl_col):
    words = mwl_col.split()
    count = 0
    for word in words:
        if len(word) > 6:
            count += 1
    return count

In [23]:
mwl['word_count'] = mwl['Word'].apply(countWords)

In [24]:
mwl.head(5)

,Word,word_count
0,x,0
1,What,0
2,frac,0
3,f,0
4,b,0


In [25]:
mwl.shape

(2258, 2)

In [26]:
mwl_complex = mwl.loc[mwl['word_count'] == 1]

In [27]:
mwl_complex.head()

,Word,word_count
20,Express,1
21,integers,1
22,equation,1
25,positive,1
30,numbers,1


In [28]:
mwl_complex.tail()

,Word,word_count
2248,calculates,1
2251,completes,1
2253,assignment,1
2254,rightarrow,1
2256,farther,1


**Here we defined that complex words are those words which are more than 6 characters**

In [29]:
mwl_complex.shape

(880, 2)

Extracting math vocab words from the column 'Problem'

In [30]:
query = '|'.join(list)
df['word'] = df['Problem '].str.findall(r'\b({})\b'.format(query))

print(df)

                                               Problem     Level   sent_sum  \
0     There are constants    alpha  and    beta  suc...  Veryhigh         2   
1     The length of the segment between the points  ...  Veryhigh         2   
2     Chewbacca has 20 pieces of cherry gum and 30 p...  Veryhigh         5   
3     For some constants  a  and  b,  let   [f(x) = ...  Veryhigh         3   
4     Calculate the sum  1 + 3 + 5 +   cdots + 15 + ...       Low         1   
...                                                 ...       ...       ...   
1739  What is the slope of the line passing through ...       Low         1   
1740  Evaluate  i^{11} + i^{16} + i^{21} + i^{26} + ...      High         1   
1741  If  a  is a constant such that  4x^2 - 12x + a...  Moderate         1   
1742  Given  a   star b = a^2 + 2ab + b^2 , what is ...  Moderate         1   
1743  The graph of the function  f(x)  is shown belo...  Veryhigh         3   

                                                   

In [31]:
df.head()

,Problem,Level,sent_sum,word
0,There are constants alpha and beta suc...,Veryhigh,2,"[There, constants, alpha, beta, frac, x, alpha..."
1,The length of the segment between the points ...,Veryhigh,2,"[The, length, segment, points, sqrt, units, Wh..."
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Veryhigh,5,"[Chewbacca, pieces, cherry, gum, pieces, grape..."
3,"For some constants a and b, let [f(x) = ...",Veryhigh,3,"[For, constants, b, let, f, x, left, begin, ar..."
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,1,"[Calculate, sum, cdots]"


Dropping the column 'Problem' from df

In [32]:
df_word = df.drop(['Problem '], axis = 1)

In [33]:
df_word.head()

,Level,sent_sum,word
0,Veryhigh,2,"[There, constants, alpha, beta, frac, x, alpha..."
1,Veryhigh,2,"[The, length, segment, points, sqrt, units, Wh..."
2,Veryhigh,5,"[Chewbacca, pieces, cherry, gum, pieces, grape..."
3,Veryhigh,3,"[For, constants, b, let, f, x, left, begin, ar..."
4,Low,1,"[Calculate, sum, cdots]"


Creating a new column called 'liststring' which takes every element of each list in lists and creates a string with each element separated by commas.

In [34]:
df_word['liststring'] = df_word.word.apply(lambda x: ', '.join([str(i) for i in x]))

In [35]:
df_word.head()

,Level,sent_sum,word,liststring
0,Veryhigh,2,"[There, constants, alpha, beta, frac, x, alpha...","There, constants, alpha, beta, frac, x, alpha,..."
1,Veryhigh,2,"[The, length, segment, points, sqrt, units, Wh...","The, length, segment, points, sqrt, units, Wha..."
2,Veryhigh,5,"[Chewbacca, pieces, cherry, gum, pieces, grape...","Chewbacca, pieces, cherry, gum, pieces, grape,..."
3,Veryhigh,3,"[For, constants, b, let, f, x, left, begin, ar...","For, constants, b, let, f, x, left, begin, arr..."
4,Low,1,"[Calculate, sum, cdots]","Calculate, sum, cdots"


Dropping of the column 'word'

In [36]:
df_liststring = df_word.drop(['word'], axis = 1)

In [37]:
df_liststring.head()

,Level,sent_sum,liststring
0,Veryhigh,2,"There, constants, alpha, beta, frac, x, alpha,..."
1,Veryhigh,2,"The, length, segment, points, sqrt, units, Wha..."
2,Veryhigh,5,"Chewbacca, pieces, cherry, gum, pieces, grape,..."
3,Veryhigh,3,"For, constants, b, let, f, x, left, begin, arr..."
4,Low,1,"Calculate, sum, cdots"


Removing the commas from the column liststring.

In [38]:
df_liststring['liststring']=df_liststring['liststring'].str.replace(',','')

In [39]:
df_liststring.head(10)

,Level,sent_sum,liststring
0,Veryhigh,2,There constants alpha beta frac x alpha x beta...
1,Veryhigh,2,The length segment points sqrt units What prod...
2,Veryhigh,5,Chewbacca pieces cherry gum pieces grape gum S...
3,Veryhigh,3,For constants b let f x left begin array cl ax...
4,Low,1,Calculate sum cdots
5,High,3,What number units area circle center P passing...
6,High,1,Find positive value n equation nx exactly one ...
7,Moderate,1,Find leading coefficient polynomial x x x x si...
8,Veryhigh,1,Find greatest frac sqrt sqrt
9,High,3,The Bank Springfield Super High Yield savings ...


Function to count number of words sum in the liststring.

In [40]:
def count_words(df_col):
  count = 0
  for i in df_col:
    for j in i:
      count = count+1
  return count

In [41]:
df_liststring['word_sum'] = df_liststring['liststring'].apply(count_words)

In [42]:
df_liststring.head()

,Level,sent_sum,liststring,word_sum
0,Veryhigh,2,There constants alpha beta frac x alpha x beta...,71
1,Veryhigh,2,The length segment points sqrt units What prod...,65
2,Veryhigh,5,Chewbacca pieces cherry gum pieces grape gum S...,263
3,Veryhigh,3,For constants b let f x left begin array cl ax...,122
4,Low,1,Calculate sum cdots,19


Function to count number of word length or word count in the liststring.

In [43]:
def word_length(df_col):
  number_of_words = len(df_col.split())
  # print(df_col.split())
  return number_of_words

In [44]:
df_liststring['word_count'] = df_liststring['liststring'].apply(word_length)

In [45]:
df_liststring.head()

,Level,sent_sum,liststring,word_sum,word_count
0,Veryhigh,2,There constants alpha beta frac x alpha x beta...,71,15
1,Veryhigh,2,The length segment points sqrt units What prod...,65,10
2,Veryhigh,5,Chewbacca pieces cherry gum pieces grape gum S...,263,44
3,Veryhigh,3,For constants b let f x left begin array cl ax...,122,31
4,Low,1,Calculate sum cdots,19,3


Function to count syllables

Counted number of syllables.

In [46]:
def syllable_count(df_col):
    df_col = df_col.lower()
    count = 0
    vowels = "aeiou"
    if df_col[0] in vowels:
        count += 1
    for index in range(1, len(df_col)):
        if df_col[index] in vowels and df_col[index - 1] not in vowels:
            count += 1
    if df_col.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

In [47]:
df_liststring['syllables'] = df_liststring['liststring'].apply(syllable_count)

In [48]:
df_liststring.head()

,Level,sent_sum,liststring,word_sum,word_count,syllables
0,Veryhigh,2,There constants alpha beta frac x alpha x beta...,71,15,19
1,Veryhigh,2,The length segment points sqrt units What prod...,65,10,15
2,Veryhigh,5,Chewbacca pieces cherry gum pieces grape gum S...,263,44,70
3,Veryhigh,3,For constants b let f x left begin array cl ax...,122,31,23
4,Low,1,Calculate sum cdots,19,3,6


COUNT OF POLY SYLLABLES

In [49]:
def count_poly_syllables(df_col):
  total_count = 0
  df_col = df_col.split()
  for word in df_col: 
    #print(word)
    temp = syllable_count(word)
    if temp>=3:
      total_count = total_count+temp
  return total_count

In [50]:
df_liststring['poly_syllables'] = df_liststring['liststring'].apply(count_poly_syllables)

In [51]:
df_liststring.head()

,Level,sent_sum,liststring,word_sum,word_count,syllables,poly_syllables
0,Veryhigh,2,There constants alpha beta frac x alpha x beta...,71,15,19,0
1,Veryhigh,2,The length segment points sqrt units What prod...,65,10,15,0
2,Veryhigh,5,Chewbacca pieces cherry gum pieces grape gum S...,263,44,70,6
3,Veryhigh,3,For constants b let f x left begin array cl ax...,122,31,23,0
4,Low,1,Calculate sum cdots,19,3,6,3


Gunning Fog Index

count_words = Numbers of characters in sentences.

word_length = Number of words in sentences.

In [52]:
# 0.4[ (words/sentence) + 100(complex words/words) ]
GF = []
for i in df.iloc[:,0]:
  w = word_length(i)
  c = countWords(i)
  s = count_sentences(i)
  gf_score = (w/s) + (100*c/w)
  GF.append(0.4*gf_score)

In [53]:
len(GF)

1744

Flesch - Kincaid Readability Index

count_words = Numbers of characters in sentences.

word_length = Number of words in sentences.

In [54]:
#Formula: 206.835 - 1.015(total words/total sentences) - 84.6(total syllables/total words)
FK = []
for i in df.iloc[:,0]:
  w = word_length(i)
  syl = syllable_count(i)
  s = count_sentences(i)
  fk_score = (1.015*w/s) + (84.6*syl/w)
  fk_score = 206.835 - fk_score 
  FK.append(fk_score)

In [55]:
len(FK)

1744

SMOG (Simple Measure of Gobbledygook) Index

In [56]:
# Fromula: 3.1291 + 1.0430(sqrt(number of polysyllables * 30 /number of sentences))
SMOG = []
for i in df.iloc[:,0]:
  syl = count_poly_syllables(i)
  s = count_sentences(i)
  smog_score = np.sqrt(30*syl/s)
  smog_score = smog_score * 1.0430
  SMOG.append(3.1291+smog_score)

In [57]:
len(SMOG)

1744

Automated Readability Index

count_words = Numbers of characters in sentences.

word_length = Number of words in sentences.

In [58]:
AR = []
for i in df.iloc[:,0]:
  w = word_length(i)
  c = count_words(i)
  s = count_sentences(i)
  ar_score = (4.71*c/w) + (0.5*w/s) - 21.43
  AR.append(ar_score)

In [59]:
len(AR)

1744

Coleman-Liau Index

count_words = Numbers of characters in sentences.

word_length = Number of words in sentences.

In [60]:
#Formula: 0.0588(Avg characters per 100 words) - 15.8 - 0.296(average number of sentences per 100 words)
CL = []
for i in df.iloc[:,0]:
  w = word_length(i)
  c = count_words(i)
  s = count_sentences(i)
  term1 = 0.0588*c*100/w #c:w :: ?:100
  term3 = 0.296*s*100/w #s:w :: ?:100
  cl_score = term1-15.8-term3
  CL.append(cl_score)

In [61]:
len(CL)

1744

In [62]:
df_liststring["GF"]=GF
df_liststring["FK"]=FK
df_liststring["SMOG"]=SMOG
df_liststring["AR"]=AR
df_liststring["CL"]=CL
df_liststring.head(5)

,Level,sent_sum,liststring,word_sum,word_count,syllables,poly_syllables,GF,FK,SMOG,AR,CL
0,Veryhigh,2,There constants alpha beta frac x alpha x beta...,71,15,19,0,12.221053,85.876711,3.129100,19.512632,26.267368
1,Veryhigh,2,The length segment points sqrt units What prod...,65,10,15,0,12.742857,101.982143,3.129100,10.129286,12.745714
2,Veryhigh,5,Chewbacca pieces cherry gum pieces grape gum S...,263,44,70,6,9.936790,78.636444,9.387100,10.161852,11.700247
3,Veryhigh,3,For constants b let f x left begin array cl ax...,122,31,23,0,13.066667,129.006333,11.855464,9.982333,11.236000
4,Low,1,Calculate sum cdots,19,3,6,3,8.666667,152.130000,13.023867,1.770000,1.826667


In [63]:
df_liststring.drop(df_liststring.columns[[2]], axis=1, inplace=True)
df_liststring.head()

,Level,sent_sum,word_sum,word_count,syllables,poly_syllables,GF,FK,SMOG,AR,CL
0,Veryhigh,2,71,15,19,0,12.221053,85.876711,3.129100,19.512632,26.267368
1,Veryhigh,2,65,10,15,0,12.742857,101.982143,3.129100,10.129286,12.745714
2,Veryhigh,5,263,44,70,6,9.936790,78.636444,9.387100,10.161852,11.700247
3,Veryhigh,3,122,31,23,0,13.066667,129.006333,11.855464,9.982333,11.236000
4,Low,1,19,3,6,3,8.666667,152.130000,13.023867,1.770000,1.826667


In [64]:
import scipy.stats as stats

from statsmodels.miscmodels.ordinal_model import OrderedModel

**Probit ordinal regression:**

In [ ]:
mod_prob = OrderedModel(df_liststring['Level'],
                        df_liststring[['sent_sum', 'word_sum', 'word_count', 'syllables', 'poly_syllables', 'GF', 'FK', 'SMOG', 'AR', 'CL']],
                        distr='probit')

res_prob = mod_prob.fit(method='bfgs')
res_prob.summary() 

In [ ]:
df_liststring.iloc[:,0].replace(["Verylow", "Low", "Moderate", "High", "Veryhigh"],
                        [1, 2, 3, 4, 5], inplace=True)

In [68]:
y=df_liststring['Level '] 

In [69]:
df_liststring.drop(df_liststring.columns[[0]], axis=1, inplace=True)
df_liststring.head()

,sent_sum,word_sum,word_count,syllables,poly_syllables,GF,FK,SMOG,AR,CL
0,2,71,15,19,0,12.221053,85.876711,3.129100,19.512632,26.267368
1,2,65,10,15,0,12.742857,101.982143,3.129100,10.129286,12.745714
2,5,263,44,70,6,9.936790,78.636444,9.387100,10.161852,11.700247
3,3,122,31,23,0,13.066667,129.006333,11.855464,9.982333,11.236000
4,1,19,3,6,3,8.666667,152.130000,13.023867,1.770000,1.826667


In [65]:
df_liststring.shape

(1744, 10)

In [66]:
#num_attr = ['sent_sum', 'word_sum', 'word_count', 'syllables', 'poly_syllables', 'GF', 'FK', 'SMOG', 'AR', 'CL']

In [67]:
num_attr = ['GF', 'FK', 'SMOG', 'AR', 'CL']

Train Test Split

In [ ]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(df_liststring, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 10), (524, 10))

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train[num_attr])

StandardScaler()

In [ ]:
X_train_std = scaler.transform(X_train[num_attr])
X_test_std = scaler.transform(X_test[num_attr])

In [ ]:
print(X_train_std.shape)
print(X_test_std.shape)

(1220, 5)
(524, 5)


In [ ]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

5    25.081967
4    23.114754
3    22.213115
2    19.672131
1     9.918033
Name: Level , dtype: float64


5    24.809160
3    23.091603
4    22.137405
2    19.083969
1    10.877863
Name: Level , dtype: float64


Model Building

In [ ]:
def perceptron_classifier (X_train_std, y_train, X_test_std, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train_std, y_train)
  predictions_train = p.predict(X_train_std)
  predictions_test = p.predict(X_test_std)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train_std, y_train, X_test_std, y_test) 

Perceptron F1 score: 0.2545905373982978


**Decision Tree**

In [ ]:
clf_dt = DecisionTreeClassifier(max_depth=5)

In [ ]:
clf_dt.fit(X_train_std,y_train)

DecisionTreeClassifier(max_depth=5)

In [ ]:
y_pred_train = clf_dt.predict(X_train_std)
y_pred_test = clf_dt.predict(X_test_std)

In [ ]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [ ]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [ ]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.39918,0.39918,0.460275,0.370432,0.265267,0.265267,0.240582,0.225947


**Random Forest**

In [ ]:
clf_rf = RandomForestClassifier()

In [ ]:
clf_rf.fit(X=X_train_std, y=y_train)

RandomForestClassifier()

In [ ]:
y_pred_train1 = clf_rf.predict(X_train_std)
y_pred_test1 = clf_rf.predict(X_test_std)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    0.9752    0.9874       121
           2     0.9794    0.9917    0.9855       240
           3     0.9853    0.9889    0.9871       271
           4     0.9929    0.9894    0.9911       282
           5     0.9935    0.9935    0.9935       306

    accuracy                         0.9893      1220
   macro avg     0.9902    0.9877    0.9889      1220
weighted avg     0.9894    0.9893    0.9893      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2857    0.1754    0.2174        57
           2     0.3056    0.3300    0.3173       100
           3     0.3010    0.2562    0.2768       121
           4     0.3063    0.2931    0.2996       116
           5     0.3713    0.4769    0.4175       130

    accuracy                         0.3244       524


In [ ]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.399180,0.399180,0.460275,0.370432,0.265267,0.265267,0.240582,0.225947
1,RandomForest,0.989344,0.989344,0.989401,0.989348,0.324427,0.324427,0.318805,0.318012


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train_std, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train_std)
y_pred_test2 = clf_adaboost.predict(X_test_std)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4423    0.1901    0.2659       121
           2     0.3678    0.2667    0.3092       240
           3     0.3164    0.2989    0.3074       271
           4     0.3246    0.3085    0.3164       282
           5     0.4043    0.6209    0.4897       306

    accuracy                         0.3648      1220
   macro avg     0.3711    0.3370    0.3377      1220
weighted avg     0.3629    0.3648    0.3514      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.1667    0.0526    0.0800        57
           2     0.1875    0.1500    0.1667       100
           3     0.2581    0.1983    0.2243       121
           4     0.2562    0.2672    0.2616       116
           5     0.3113    0.5077    0.3860       130

    accuracy                         0.2653       524


In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.399180,0.399180,0.460275,0.370432,0.265267,0.265267,0.240582,0.225947
1,RandomForest,0.989344,0.989344,0.989401,0.989348,0.324427,0.324427,0.318805,0.318012
2,Adaboost,0.364754,0.364754,0.362941,0.351428,0.265267,0.265267,0.247455,0.245970


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train_std, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train_std)
y_pred_test3 = clf_gbm.predict(X_test_std)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9059    0.6364    0.7476       121
           2     0.8556    0.6417    0.7333       240
           3     0.7303    0.7196    0.7249       271
           4     0.7535    0.7695    0.7614       282
           5     0.6700    0.8758    0.7592       306

    accuracy                         0.7467      1220
   macro avg     0.7830    0.7286    0.7453      1220
weighted avg     0.7626    0.7467    0.7459      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2000    0.1228    0.1522        57
           2     0.2529    0.2200    0.2353       100
           3     0.2650    0.2562    0.2605       121
           4     0.3246    0.3190    0.3217       116
           5     0.3801    0.5000    0.4319       130

    accuracy                         0.3092       524


In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.399180,0.399180,0.460275,0.370432,0.265267,0.265267,0.240582,0.225947
1,RandomForest,0.989344,0.989344,0.989401,0.989348,0.324427,0.324427,0.318805,0.318012
2,Adaboost,0.364754,0.364754,0.362941,0.351428,0.265267,0.265267,0.247455,0.245970
3,GBM,0.746721,0.746721,0.762595,0.745852,0.309160,0.309160,0.297350,0.299985


**SVM Model**

In [ ]:
linear_svm = SVC(kernel='linear', C=1)

In [ ]:
linear_svm.fit(X=X_train_std, y= y_train)

SVC(C=1, kernel='linear')

In [ ]:
y_pred_train4 = linear_svm.predict(X_train_std)
y_pred_test4 = linear_svm.predict(X_test_std)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.0000    0.0000    0.0000       121
           2     0.3056    0.0458    0.0797       240
           3     0.2507    0.3137    0.2787       271
           4     0.2336    0.2908    0.2591       282
           5     0.3360    0.5425    0.4150       306

    accuracy                         0.2820      1220
   macro avg     0.2252    0.2386    0.2065      1220
weighted avg     0.2541    0.2820    0.2416      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.0000    0.0000    0.0000        57
           2     0.1739    0.0400    0.0650       100
           3     0.2042    0.2397    0.2205       121
           4     0.2516    0.3448    0.2909       116
           5     0.3250    0.5000    0.3939       130

    accuracy                         0.2634       524


In [ ]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.399180,0.399180,0.460275,0.370432,0.265267,0.265267,0.240582,0.225947
1,RandomForest,0.989344,0.989344,0.989401,0.989348,0.324427,0.324427,0.318805,0.318012
2,Adaboost,0.364754,0.364754,0.362941,0.351428,0.265267,0.265267,0.247455,0.245970
3,GBM,0.746721,0.746721,0.762595,0.745852,0.309160,0.309160,0.297350,0.299985
4,SVM,0.281967,0.281967,0.254090,0.241563,0.263359,0.263359,0.216670,0.225470


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train_std, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train_std)
y_pred_test5 = knn.predict(X_test_std)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4000    0.3802    0.3898       121
           2     0.4239    0.4875    0.4535       240
           3     0.4120    0.4576    0.4336       271
           4     0.5000    0.4433    0.4699       282
           5     0.5468    0.4967    0.5205       306

    accuracy                         0.4623      1220
   macro avg     0.4565    0.4530    0.4535      1220
weighted avg     0.4673    0.4623    0.4634      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.1522    0.1228    0.1359        57
           2     0.2314    0.2800    0.2534       100
           3     0.2137    0.2066    0.2101       121
           4     0.2393    0.2414    0.2403       116
           5     0.3659    0.3462    0.3557       130

    accuracy                         0.2538       524


In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.399180,0.399180,0.460275,0.370432,0.265267,0.265267,0.240582,0.225947
1,RandomForest,0.989344,0.989344,0.989401,0.989348,0.324427,0.324427,0.318805,0.318012
2,Adaboost,0.364754,0.364754,0.362941,0.351428,0.265267,0.265267,0.247455,0.245970
3,GBM,0.746721,0.746721,0.762595,0.745852,0.309160,0.309160,0.297350,0.299985
4,SVM,0.281967,0.281967,0.254090,0.241563,0.263359,0.263359,0.216670,0.225470
5,KNN,0.462295,0.462295,0.467287,0.463369,0.253817,0.253817,0.253799,0.253114


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train_std, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train_std)
y_pred_test6 = nb.predict(X_test_std)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3333    0.0331    0.0602       121
           2     0.2754    0.2375    0.2550       240
           3     0.2000    0.0148    0.0275       271
           4     0.2523    0.6950    0.3702       282
           5     0.3235    0.2157    0.2588       306

    accuracy                         0.2680      1220
   macro avg     0.2769    0.2392    0.1943      1220
weighted avg     0.2711    0.2680    0.2127      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.0000    0.0000    0.0000        57
           2     0.2178    0.2200    0.2189       100
           3     0.1667    0.0083    0.0157       121
           4     0.2448    0.7069    0.3636       116
           5     0.3766    0.2231    0.2802       130

    accuracy                         0.2557       524


In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores) 
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.399180,0.399180,0.460275,0.370432,0.265267,0.265267,0.240582,0.225947
1,RandomForest,0.989344,0.989344,0.989401,0.989348,0.324427,0.324427,0.318805,0.318012
2,Adaboost,0.364754,0.364754,0.362941,0.351428,0.265267,0.265267,0.247455,0.245970
3,GBM,0.746721,0.746721,0.762595,0.745852,0.309160,0.309160,0.297350,0.299985
4,SVM,0.281967,0.281967,0.254090,0.241563,0.263359,0.263359,0.216670,0.225470
5,KNN,0.462295,0.462295,0.467287,0.463369,0.253817,0.253817,0.253799,0.253114
6,GNB,0.268033,0.268033,0.271111,0.212723,0.255725,0.255725,0.227679,0.195426


In [63]:
import scipy.stats as stats

from statsmodels.miscmodels.ordinal_model import OrderedModel

Probit ordinal regression:

In [ ]:
mod_prob = OrderedModel(df_liststring['Level '],
                        df_liststring[['sent_sum', 'word_sum', 'word_count', 'syllables', 'poly_syllables', 'GF', 'FK', 'SMOG', 'AR', 'CL']],
                        distr='probit')

res_prob = mod_prob.fit(method='bfgs')
res_prob.summary()